In [3]:
from keras.models import load_model

Using TensorFlow backend.


In [4]:
import os
import numpy as np
from PIL import Image
import copy
import pickle

In [5]:
trial_model=load_model("C:\\Users\\Om\\Desktop\\Cerberus\\Utilities\\VGGFace.h5")

C:\Users\Om\Anaconda3\envs\opencv\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
def features(featmodel, crpimg, transform=False): #extract face feature vector
    
    # transform=True seems more robust but I think the RGB channels are not in right order
    
    imarr = np.array(crpimg).astype(np.float32)

    if transform:
        imarr[:,:,0] -= 129.1863
        imarr[:,:,1] -= 104.7624
        imarr[:,:,2] -= 93.5940        
        aux = copy.copy(imarr)
    imarr = np.expand_dims(imarr, axis=0)
    output= featmodel.predict(imarr)[0,:]
    return output

In [7]:
def feature_extractor(inpDir,personID,wordID,timestep): #access directory and extract face vector     
        
    sample_space=[] #for appending different utterances
    feature_sequence = [] # for appending different timesteps
    inpDir=inpDir #input dataset
    linpDir = os.listdir(inpDir) #list all directories in dataset
    #print(linpDir)
    personStr= linpDir[personID]
    personFolder = '%s\\%s\\words' % (inpDir,personStr) #opening words folder
    lpersonFolder = os.listdir(personFolder)
    wordID= lpersonFolder[wordID]
    wordFolder = '%s\\%s' % (personFolder,wordID)
    lwordFolder = os.listdir(wordFolder)
    i=0
    for utterance in lwordFolder:
        
        utterFolder= '%s\\%s' % (wordFolder,utterance)
        lutterFolder = os.listdir(utterFolder)
        colourFolder ='%s\\%s' % (utterFolder,lutterFolder[0])
        lcolourFolder = os.listdir(colourFolder)
        frame = lcolourFolder[timestep]
        i += 1
        image= "%s\\%s" % (colourFolder,frame) 
        im=Image.open(image)
        im = im.resize((224,224))
        feature_vector = features(trial_model,im, transform=True).reshape((1,1, 1, 2622))
        if i==1 :
            feature_sequence=feature_vector #done because of need of same dim. for concatenation
        else:
            feature_sequence = np.concatenate((feature_sequence,feature_vector),axis=0)
            
    feature_sequence1 = feature_sequence[:,0,0,:]
    return feature_sequence1

In [8]:
def dict_convertor(array):
    convertor={}
    for i in range(len(array)):
        convertor[i]=array[i]
    return convertor

In [9]:
def reference(dict, length):
    ref = [None]*len(dict[1])

    for i in range(0, len(dict[1])):
        ref[i] = 0
    for i in range(len(dict[1])):
        for j in range(1, length):
            ref[i] = ref[i] + dict[j][i]
        ref[i] = ref[i]/length

    return ref

In [17]:
def append(ref_dict, arr):
    ref_dict.update({(len(ref_dict)+1): arr})
    return ref_dict

In [11]:
people_list_for_training=[0,1,2,3,4,5,6,10,11,12]

In [18]:
def people_extractor(inpDir,person_list):
    People_dictionary={}
    for people in range(0,len(person_list)):
        feature_sequence=feature_extractor(inpDir,person_list[people],0,3)
        convertor=dict_convertor(feature_sequence)
        average=[]
        average=reference(convertor,len(convertor))
        average=np.array((average))
        feat_array=average.reshape(1,2622)
        People_dictionary=append(People_dictionary,feat_array)
    return People_dictionary

In [13]:
people_dictionary=people_extractor('C:\\Users\\Om\\Desktop\\dataset',people_list_for_training)

[[6.03537488e-05 2.69179969e-05 9.28771387e-05 ... 2.78823231e-06
  3.78060906e-04 7.20144567e-04]]
0
[[6.40833334e-03 2.01818076e-03 1.78118343e-04 ... 2.68819867e-05
  4.46896930e-04 3.64193419e-04]]
1
[[4.12250037e-04 3.81355553e-04 4.94044976e-05 ... 4.84345772e-05
  6.46252511e-04 1.19746818e-04]]
2
[[1.50350405e-04 1.88538158e-05 5.56068490e-05 ... 1.12473192e-05
  5.29548946e-04 2.15044238e-04]]
3
[[1.48100094e-04 4.68411972e-05 3.28964730e-05 ... 5.85897794e-06
  5.41940581e-05 1.73809101e-04]]
4
[[5.96197996e-04 3.87119139e-04 9.21455463e-06 ... 3.80263485e-06
  1.41481093e-04 2.78760788e-04]]
5
[[2.47419154e-04 6.48256135e-05 1.71964975e-05 ... 7.02096831e-06
  8.09318869e-04 9.48089350e-04]]
6
[[2.19689102e-04 3.94609806e-05 6.39009578e-04 ... 3.57697432e-05
  2.37491962e-04 1.02846096e-04]]
7
[[5.82142175e-05 3.82201528e-05 6.67964287e-05 ... 1.32054201e-05
  1.01038714e-04 3.44952939e-04]]
8
[[2.03583384e-03 7.88221383e-05 2.92029144e-04 ... 3.75044157e-05
  1.05454317e-04

In [16]:
output = open('training_dict.pkl', 'wb')
pickle.dump(people_dictionary, output)
output.close()